In [1]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, hstack

In [3]:

from sklearn.preprocessing import LabelEncoder
import numpy as np
# 读取数据集
# data = pd.read_csv('train.csv')
data = np.load("/home/train.pkl", allow_pickle=True)
data

vectorizer = CountVectorizer()

vecto = vectorizer.fit_transform(data["text_cleaned"])
csr = csr_matrix(vecto)




In [4]:
import re
def get_urls(text:str):
    pattern = re.compile(r"http\S+")
    urls = pattern.findall(text)
    return urls

def get_mentions(text:str):
    pattern = re.compile(r"@\S+")
    mentions = pattern.findall(text)
    return mentions

data["urls"] = data["text_cleaned"].apply(get_urls)
data["mentions"] = data["text_cleaned"].apply(get_mentions)


In [5]:

le = LabelEncoder()

keywords  = le.fit_transform(data["keyword"].fillna(""))
locations = le.fit_transform(data["location"].fillna(""))
mentions  = le.fit_transform(data["mentions"].apply(lambda x: " ".join(x)).fillna(""))
urls      = le.fit_transform(data["urls"].apply(lambda x: " ".join(x)).fillna(""))





In [6]:



csr = hstack((csr, csr_matrix( keywords.reshape(-1, 1))))
csr = hstack((csr, csr_matrix(locations.reshape(-1, 1))))
csr = hstack((csr, csr_matrix(mentions.reshape(-1, 1))))
csr = hstack((csr, csr_matrix(urls.reshape(-1, 1))))
# print first 5 rows of csr_matrix
print(csr.toarray().shape)

(7613, 16885)


In [7]:

X_train, X_test, y_train, y_test = train_test_split(csr, data['target_relabeled'], test_size=0.2, random_state=42)



In [8]:

# 训练模型
# ramdom forest
from lazypredict.Supervised import LazyClassifier


clf = LazyClassifier(predictions=True)
models ,predictions = clf.fit(X_train.toarray(), X_test.toarray(), y_train, y_test)

models


100%|██████████| 29/29 [59:07<00:00, 122.33s/it] 


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
BernoulliNB,0.82,0.80,0.80,0.81,7.79
NearestCentroid,0.81,0.80,0.80,0.81,7.36
ExtraTreesClassifier,0.80,0.78,0.78,0.80,118.43
LGBMClassifier,0.79,0.77,0.77,0.79,10.35
XGBClassifier,0.79,0.77,0.77,0.79,40.20
RandomForestClassifier,0.79,0.77,0.77,0.79,57.86
NuSVC,0.79,0.77,0.77,0.79,685.67
Perceptron,0.77,0.76,0.76,0.77,10.55
LogisticRegression,0.76,0.75,0.75,0.76,10.66


In [9]:
import pickle

with open("/home/model_fin.pkl", "wb") as f:
    pickle.dump(models, f)